In [5]:
# Ensure the project root is in PATH.
import sys

sys.path.append("../")
# All imports of our code are relative to the project root.

from backtester.engine import Backtester
from backtester.datamodel import TradingState, OrderDepth, Order, Listing
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import sys
import os
from backtester.log import Log

from collections import defaultdict


# concatenates multiple days of historical data into 1.
# drops day column and replaces it with continuous timestamps.
# i.e. day -1 timestamp 0 becomes just timestamp 1,000,000
def concatenate_historical_data(data: list[pd.DataFrame]) -> pd.DataFrame:
    output = data[0]

    for i in range(1, len(data), 1):
        timeshift = output.iloc[-1]["timestamp"] + 100  # 100 for next day
        next_day_copy = data[i].copy()
        next_day_copy["timestamp"] += timeshift

        output = pd.concat([output, next_day_copy])

    return output


def get_time_part(df: pd.DataFrame, l, h) -> pd.DataFrame:
    dfret = df.copy()
    dfret = dfret[(dfret["timestamp"] >= l) & (dfret["timestamp"] < h)].reset_index(drop=True)
    return dfret

In [6]:
CROISSANTS = "CROISSANTS"
DJEMBES = "DJEMBES"
JAMS = "JAMS"
KELP = "KELP"
PICNIC_BASKET1 = "PICNIC_BASKET1"
PICNIC_BASKET2 = "PICNIC_BASKET2"
RAINFOREST_RESIN = "RAINFOREST_RESIN"
SQUID_INK = "SQUID_INK"
VOLCANIC_ROCK = "VOLCANIC_ROCK"
VOLCANIC_ROCK_VOUCHER_10000 = "VOLCANIC_ROCK_VOUCHER_10000"
VOLCANIC_ROCK_VOUCHER_10250 = "VOLCANIC_ROCK_VOUCHER_10250"
VOLCANIC_ROCK_VOUCHER_10500 = "VOLCANIC_ROCK_VOUCHER_10500"
VOLCANIC_ROCK_VOUCHER_9500 = "VOLCANIC_ROCK_VOUCHER_9500"
VOLCANIC_ROCK_VOUCHER_9750 = "VOLCANIC_ROCK_VOUCHER_9750"
MAGNIFICENT_MACARONS = "MAGNIFICENT_MACARONS"

market_data_round_5_day_2 = pd.read_csv(os.path.join("..", "data", "round5", "prices_round_5_day_2.csv"), sep=";")
market_data_round_5_day_3 = pd.read_csv(os.path.join("..", "data", "round5", "prices_round_5_day_3.csv"), sep=";")
market_data_round_5_day_4 = pd.read_csv(os.path.join("..", "data", "round5", "prices_round_5_day_4.csv"), sep=";")

trades_round_5_day_2 = pd.read_csv(os.path.join("..", "data", "round5", "trades_round_5_day_2.csv"), sep=";")
trades_round_5_day_3 = pd.read_csv(os.path.join("..", "data", "round5", "trades_round_5_day_3.csv"), sep=";")
trades_round_5_day_4 = pd.read_csv(os.path.join("..", "data", "round5", "trades_round_5_day_4.csv"), sep=";")

observations_round_5_day_2 = pd.read_csv(os.path.join("..", "data", "round5", "observations_round_5_day_2.csv"), sep=",")
observations_round_5_day_3 = pd.read_csv(os.path.join("..", "data", "round5", "observations_round_5_day_3.csv"), sep=",")
observations_round_5_day_4 = pd.read_csv(os.path.join("..", "data", "round5", "observations_round_5_day_4.csv"), sep=",")

market_data_round_5_all3days = concatenate_historical_data([market_data_round_5_day_2, market_data_round_5_day_3, market_data_round_5_day_4])
trades_round_5_all3days = concatenate_historical_data([trades_round_5_day_2, trades_round_5_day_3, trades_round_5_day_4])
observations_round_5_all3days = concatenate_historical_data([observations_round_5_day_2, observations_round_5_day_3, observations_round_5_day_4])

round4finallog = Log.from_file("round4final.log")


In [7]:
md = market_data_round_5_day_4.copy()
th = trades_round_5_day_4.copy()
obs = observations_round_5_day_4.copy()

In [23]:
plt.figure(figsize=(12, 6))


market_data = market_data_round_5_day_4.copy()
market_data = market_data[market_data['product'] == PICNIC_BASKET1]


import pandas as pd
import plotly.express as px

# Melt bid/ask price and volume columns into long format
def prepare_liquidity_df(market_data):
    n = len(market_data)

    bids = pd.DataFrame({
        "timestamp": market_data["timestamp"].values.repeat(3),
        "side": ["buy"] * 3 * n,
        "price": pd.concat([
            market_data["bid_price_1"],
            market_data["bid_price_2"],
            market_data["bid_price_3"]
        ], ignore_index=True),
        "volume": pd.concat([
            market_data["bid_volume_1"],
            market_data["bid_volume_2"],
            market_data["bid_volume_3"]
        ], ignore_index=True)
    })

    asks = pd.DataFrame({
        "timestamp": market_data["timestamp"].values.repeat(3),
        "side": ["sell"] * 3 * n,
        "price": pd.concat([
            market_data["ask_price_1"],
            market_data["ask_price_2"],
            market_data["ask_price_3"]
        ], ignore_index=True),
        "volume": pd.concat([
            market_data["ask_volume_1"],
            market_data["ask_volume_2"],
            market_data["ask_volume_3"]
        ], ignore_index=True)
    })

    df_long = pd.concat([bids, asks], ignore_index=True)
    df_long.dropna(subset=["price", "volume"], inplace=True)
    return df_long


# Prepare long-format data
df_liquidity = prepare_liquidity_df(market_data)

# Plot with Plotly
fig = px.scatter(
    df_liquidity,
    x="timestamp",
    y="price",
    size="volume",
    color="side",
    title="Liquidity Over Time (Bid/Ask at Price Levels)",
    labels={"timestamp": "Time", "price": "Price"},
    opacity=0.6,
)

fig.update_layout(height=600, width=1000)
fig.show()


<Figure size 1200x600 with 0 Axes>

In [26]:
spread = market_data['ask_price_1'] - market_data['bid_price_1']

np.mean(spread), np.std(spread)

(9.176, 1.933810745652221)